**Add your LiU-ID here:**
* <liuid 1>
* <liuid 2>

## **Quick introduction to jupyter notebooks**
* Each cell in this notebook contains either code or text.
* You can run a cell by pressing Ctrl-Enter, or run and advance to the next cell with Shift-Enter.
* Code cells will print their output, including images, below the cell. Running it again deletes the previous output, so be careful if you want to save some results.
* You don't have to rerun all cells to test changes, just rerun the cell you have made changes to. Some exceptions might apply, for example if you overwrite variables from previous cells, but in general this will work.
* If all else fails, use the "Kernel" menu and select "Restart Kernel and Clear All Output". You can also use this menu to run all cells.
* A useful debug tool is the console. You can right-click anywhere in the notebook and select "New console for notebook". This opens a python console which shares the environment with the notebook, which let's you easily print variables or test commands.

### **Setup**

In [ ]:
# Automatically reload modules when changed
%reload_ext autoreload
%autoreload 2
# Plot figures "inline" with other output
%matplotlib inline

# Most important package
import numpy as np

# The reinforcement learning environment
from gridworld import GridWorld

# Configure nice figures
from matplotlib import pyplot as plt
plt.rcParams['figure.facecolor']='white'
plt.rcParams['figure.figsize']=(14,7)

### ***! IMPORTANT NOTE !***

Your implementation should only use the `numpy` (`np`) module. The `numpy` module provides all the functionality you need for this assignment and makes it easier debuging your code. No other modules, e.g. `scikit-learn` or `scipy` among others, are allowed and solutions using modules other than `numpy` will be sent for re-submission. You can find everything you need about `numpy` in the official [documentation](https://numpy.org/doc/stable/).

---
## **1. Reinforcement Learning, introduction**
In the previous assignments we have explored supervised learning, in other words, methods that train a model based on known inputs and targets. This time, we will instead look at a branch of machine learning that is much closer to the intuitive notion of "learning". Reinforcement learning, or RL for short, does not work with inputs and targets, but instead learns by performing **actions** in an **environment** and observing the generated **rewards**.

RL is a very broad concept and many different algorithms have been deviced based on these general concepts of actions and rewards. Perhaps the main advantage of RL over other machine learning techniques is that we do not explicitly tell the model what the right answer is (like we have done in the previous assignments), but instead only tell the model when the desired outcome has been acheived. This might seem like the same thing at first, but the key difference is that RL allows the model to device solutions that outperform the human teacher. This is usually not possible in traditional supervised learning since the model can only get as good as the training data (the teacher). With the freedom to explore new strategies, which is inherent to RL, this is no longer true and some truly astounding results have been acheved. The most famous example is probably AlphaGo, the first computer program to beat a human expert in the board game Go. [Here is an excellent documentary](https://youtu.be/WXuK6gekU1Y), if you have some time to spare. For those of you that want a quicker and more fun example, [here is a video about RL agents playing hide and seek](https://youtu.be/kopoLzvh5jY), which very clearly demonstrates the power of RL to invent new and hidden strategies.

Of course, these examples are from the very forefront of current research in RL, and are unfortunately too complex for this assignment. We will instead work on a much simpler problem, but the core concepts that you will implement and investigate here are the same that made the above possible.

### **1.1 Getting to know the environment interface**
To do this assignment you must first get familiar with the code interface to the environment, or "World", as we will call it. You will work with a special type of environment called a **GridWorld**. The GridWorld is, as the name suggests, a world where each state is represented by a square on a grid. To create an instance of a GridWorld, run the following code. You can change the input number to select a different world. You will work with worlds 1-4, but there are other optional worlds as well, which we encourage you to explore at the end of the notebook.

In [ ]:
W = GridWorld(1)
W.init()
W.draw()

#### **<span style="color:red">Question 1:</span>**
The colored background represents the reward for entering each state. Notice that all rewards are negative. Can you think of why this is important?

#### **<span style="color:green">Answer:</span>**
\[ Your answer here \]

The **Agent** is represented by the gray square, and will traverse the environment in order to reach the **goal** state, represented by the green circle.
You can access all information you need regarding the state of the GridWorld by the methods of the World class. Here is the full list with explanations for each method:

* `getWorldSize()` - Returns a tuple with the size of each dimension in the state space. For the GridWorlds, this is the y-size and x-size of the grid.
* `getDimensionNames()` - Returns a list with the names for each dimension. This is only used to understand the world better, and should not be used to design the algorithm.
* `getActions()` - Returns a list of available actions in the form of strings. These are the only accepted values to pass to `doAction`.
* `init()` - Initializes the World. For example this resets the position of the agent in the GridWorlds. Do this at the beginning of each epoch.
* `getState()` - Returns the current state of the World, which for a GridWorld is the position of the agent.
* `doAction(act)` - Performs an action and returns a 2-tuple indicating if the actions was valid, and the corresponding reward.
* `draw(epoch, Q)` - Update any plots associated with the World. The two arguments are optional but will include more information in the plots if you provide them.

Here are some examples:

In [ ]:
W = GridWorld(1)
print("World size:", W.getWorldSize())
print("Dimension names:", W.getDimensionNames())
print("Actions:", W.getActions())

Here is an example of some actions in the first GridWorld. Read the code and output and make sure you understand how this works before proceeding. You can quickly run the cell multiple times by holding `Ctrl` and pressing `Enter` to generate a new output.

In [ ]:
W = GridWorld(1)
W.init()

# Check state
state, isTerm = W.getState()
print(f"State initialized to {state}.")

# Make action
a = "Down"
isValid, reward = W.doAction(a)
print(f"Action '{a}' was {'' if isValid else 'not '}valid and gave a reward of {reward}.")

# Check state
state, isTerm = W.getState()
print(f"State is {state} and is {('' if isTerm else 'not ')}terminal.")

# Make action
a = "Right"
isValid, reward = W.doAction(a)
print(f"Action '{a}' was {'' if isValid else 'not '}valid and gave a reward of {reward}.")

# Check state
state, isTerm = W.getState()
print(f"State is {state} and is {('' if isTerm else 'not ')}terminal.")

---
## **2. Implementing the Q-learning algorithm**
You will now implement the main algorithm of this assignment, **Q-learning**. This algorithm is powerful since it allows the simultaneous exploration of different **policies**. This is done by a state-action table **Q**, keeping track of the expected reward associated with each action in each state. By iteratively updating these estimates as we get new rewards, the policies explored by the agent eventually converges to the optimal policy. This can all be summarized in the following equation:

$$ \large Q\left(s_t,a\right) \leftarrow \underbrace{Q\left(s_t,a\right)}_{\mathrm{Old \space value}} \cdot \left(1-\alpha\right) + \alpha \cdot \underbrace{\left(r + \gamma V\left(s_{t+1}\right)\right)}_{\mathrm{New \space estimate}} $$

This defines that the value of $Q$ in a state $s_t$ for action $a$, i.e $Q\left(s_t,a\right)$, should be updated as a weighted average of the old value and a new estimate, where the weighting is based on the learning rate $\alpha \in (0,1)$. The new estimate is a combination of the reward $r$ for the action we are updating, and the estimated value $V$ of the next state $s_{t+1}$, discounted by the factor $\gamma \in (0,1]$. By increasing $\gamma$, the future value is weighted higher, which is why we say that this optimizes for long-term rewards.

### 2.1 The training function
First, you will implement the Q-learning algorithm training loop in the following function. The inputs to this function is a World object, and a dictionary for any parameters needed for the training. For example, this could be the learning rate, discount factor, number of training epochs, etc. You must decide what parameters you want to use and what to call them, but you must use this function signature. This is to make the code further down in the notebook run as intended.

A dictionary is created in python by a list of name-value pairs in curly brackets, for example `params = {"Epochs": 100, "DiscountFactor": 0.9}` and so on. You access the content of the dictionary by it's name, for example `params["DiscountFactor"]`. Using this style makes it very easy later in the notebook to try new worlds and parameter combinations, and we therefore REQUIRE that you use it.

Finally before you begin, here are some concrete tips to keep in mind while working:
* Try your code often! Jump ahead to section 3.1 to easily run the training in the first GridWorld.
* When initializing the Q-table, it will be natural to use a 2D state space since that directly matches the grid structure of the GridWorlds. This means that when you add in the action dimension, the Q-table will be 3D. You can use the `getWorldSize` and `getActions` methods of the World class to get the correct sizes for each dimension. After this, it is trivial to index into the Q-table using the syntax `Q[state][action_index]`, where `action_index` is the index of the action in the list returned by `getActions()`. Our pre-provided draw functions also assumes this structure, so make sure to follow it.
* Make sure you call the `draw` function of the world at regular intervals to see the training progress. For example after every 100th epoch.
* It can be a good idea to implement a limit to the number of steps (actions) the agent is allowed to take before the epoch is reset, to prevent getting stuck in an infinite loop.
* As part of this implementation, <span style="color:#ff6666">you must also implement the functions</span> `getpolicy` and `getvalue` in `utils.py`. When you have implemented these the `draw` function will automatically show the results of the training!


In [ ]:
def QLearning(World, params={}):
    
    # Init world and get size of dimensions
    WSize = World.getWorldSize()
    A = World.getActions()
    NA = len(A)

    # --------------------------------------------
    # === Your code here =========================
    # --------------------------------------------
    
    # Initialize the Q-matrix (use the size variables above)
    Q = ???
    
    for i in range(params["Epochs"]):
        ???
        
        # Limiting the number of steps in an epoch prevents getting stuck in infinite loops
        for j in range(params["MaxSteps"]):
            ???
        
        # Update plots with regular intervals
        if ((i+1) % params["DrawInterval"] == 0) or (i == params["Epochs"]-1):
            World.draw(epoch=(i+1), Q=Q, params=params)
    
    # ============================================
        
    return Q

### **2.2 The test function**
It's important to test the performance of the trained model. This *could* be done with some heuristic function that measures properties such as path lenghts and total rewards, but here we choose to instead use a more direct evaluation method. In the following function you should implement a test loop where you follow the optimal policy and draw the world after *each* action. Since this is code to test the trained model, you should not update Q, only use it to determine the optimal actions.

In [ ]:
def QLearningTest(W, Q, params={}):
    
    # The number of epochs is now the number of tests runs to do
    for i in range(params["Epochs"]):
        
        # Init the world and get state
        W.init()
        A = W.getActions()
        s,_ = W.getState()

        # Again we limit the number of steps to prevent infinite loops
        for j in range(params["MaxSteps"]):
            
            # --------------------------------------------
            # === Your code here =========================
            # --------------------------------------------
            
            # Choose and perform optimal action from policy
            ???

            # ============================================
            
            # Get updated state and draw
            s,isTerm = W.getState()
            W.draw(epoch=(i+1), Q=Q, params=params)
            
            # Check if goal
            if isTerm:
                break

---
## **3. Optimizing the different worlds**

In this section you will optimize the hyperparameters to train the 4 first GridWorlds. 

### **3.1 GridWorld 1**
We start with the simplest of the worlds, "Annoying block". The policy should converge without much difficulty, so use this as a test to see if your implementaion is correct. If you use a good set of hyperparameters, you can expect a rather neat policy in about 1000 epochs.

In [ ]:
W1 = GridWorld(1)

# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

Q1 = QLearning(W1, params={"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": ???, "MaxSteps": 200, "DrawInterval": 100})

# ============================================

Don't forget to run a few tests with the optimized policy to see if the solution looks reasonable.

In [ ]:
QLearningTest(W=W1, Q=Q1, params={"Epochs": 5, "MaxSteps": 100})

#### **<span style="color:red">Question 2:</span>**
1. Describe World 1.
2. What is the goal for the agent in this world?
3. What is a good choice of learning rate in this world? Motvate your answer.

#### **<span style="color:green">Answer:</span>**
\[ Your answers here \]

Now continue optimizing worlds 2-4. Note that the optimal hyperparmeters potentially are very different for each world.

### **3.2 GridWorld 2**

In [ ]:
W2 = GridWorld(2)

# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

Q2 = QLearning(W2, params={"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": ???, "MaxSteps": 200, "DrawInterval": 100})

# ============================================

Don't forget to run a few tests with the optimized policy to see if the solution looks reasonable.

In [ ]:
QLearningTest(W=W2, Q=Q2, params={"Epochs": 5, "MaxSteps": 100})

#### **<span style="color:red">Question 3:</span>**
1. Describe World 2.
2. This world has a hidden trick. Describe this trick and why this can be solved with reinforcement learning.
3. What is the goal for the agent in this world?
4. What is a good choice of learning rate in this world? Motvate your answer.
5. Compared to the optimal policy in World 1, how do we expect the optimal policy to look in this world? Motivate your answer.

#### **<span style="color:green">Answer:</span>**
\[ Your answers here \]

### **3.3 GridWorld 3**
Note that unlike World 1 and 2, the start position is not randomized in this world. The agent always starts in the same position.

In [ ]:
W3 = GridWorld(3)

# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

Q3 = QLearning(W3, params={"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": ???, "MaxSteps": 200, "DrawInterval": 100})

# ============================================

Don't forget to run a few tests with the optimized policy to see if the solution looks reasonable.

In [ ]:
QLearningTest(W=W3, Q=Q3, params={"Epochs": 5, "MaxSteps": 100})

#### **<span style="color:red">Question 4:</span>**
1. Describe World 3.
2. From the perspective of the learning algorithm, how does this world compare to World 1?
3. What is the goal for the agent in this world?
4. Is it possible to get a good policy in every state in this world? If so, which hyperparameter is particulary important to acheive this?

#### **<span style="color:green">Answer:</span>**
\[ Your answers here \]

### **3.4 GridWorld 4**
Note that unlike World 1 and 2, the start position is not randomized in this world. The agent always starts in the same position.

**Important**: In this world it is especially important to investigate the behaviour of the agent using the test function, in order to find the "trick" we are asking about in Question 5. You might think the policy looks bad, but we encourage you to run the test even if you think it's not optimal. It might give you some insight into the world behaviour.

In [ ]:
W4 = GridWorld(4)

# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

Q4 = QLearning(W4, params={"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": ???, "MaxSteps": 200, "DrawInterval": 100})

# ============================================

In [ ]:
QLearningTest(W=W4, Q=Q4, params={"Epochs": 5, "MaxSteps": 100})

#### **<span style="color:red">Question 5:</span>**
1. Describe World 4 using your own words. 
2. This world has a hidden trick. What is it, and how does this world differ from World 3?
3. What is the goal for the agent in this world?
4. What is a good choice of learning rate in this world? Motvate your answer.
5. How should we expect the optimal policy too look like? In other words, what is the optimal path from start to goal in this world? Motivate your answer.

#### **<span style="color:green">Answer:</span>**
\[ Your answers here \]

---
## **4. Investigating the effects of hyperparameters**
You will now design a series of experiments to show the impact of the three main hyperparameters - learning rate, discount factor, and exploration rate - in different environments. You are free to extend the experiments as you see fit in order to make your point in the discussions, but a recommended strategy is to try two extreme cases (low vs high values). For each parameter, there is one world in particular of the four you have already used where it is easy to show the effects we are looking for. Figuring out which worlds is part of the excercise.

### 4.1 Learning rate ($\alpha$)

In [ ]:
# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

W_LR = GridWorld(???)
Q_41_L = QLearning(W_LR, params={"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": ???, "MaxSteps": 200, "DrawInterval": 100})

# ============================================

In [ ]:
# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

Q_LR_H = QLearning(W_LR, params={"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": ???, "MaxSteps": 200, "DrawInterval": 100})

# ============================================

#### **<span style="color:red">Question 6:</span>**
Explain your experiment and results, and why you choose this world (your answers should be based on the output of the cells above).

#### **<span style="color:green">Answer:</span>**
\[ Your answer here \]

### 4.2 Discount factor ($\gamma$)

In [ ]:
# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

W_DF = GridWorld(???)
Q_DF_L = QLearning(W_DF, params={"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": ???, "MaxSteps": 200, "DrawInterval": 100})

# ============================================

In [ ]:
# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

Q_DF_H = QLearning(W_DF, params={"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": ???, "MaxSteps": 200, "DrawInterval": 100})

# ============================================

#### **<span style="color:red">Question 7:</span>**
Explain your experiment and results, and why you choose this world (your answers should be based on the output of the cells above).

#### **<span style="color:green">Answer:</span>**
\[ Your answer here \]

### 4.3 Exploration rate ($\varepsilon$)

In [ ]:
# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

W_ER = GridWorld(???)
Q_ER_L = QLearning(W_ER, params={"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": ???, "MaxSteps": 200, "DrawInterval": 100})

# ============================================

In [ ]:
# --------------------------------------------
# === Your code here =========================
# --------------------------------------------

Q_ER_H = QLearning(W_ER, params={"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": ???, "MaxSteps": 200, "DrawInterval": 100})

# ============================================

#### **<span style="color:red">Question 8:</span>**
Explain your experiment and results, and why you choose this world (your answers should be based on the output of the cells above).

#### **<span style="color:green">Answer:</span>**
\[ Your answer here \]

---
## 5. Optional gridworlds

You have now investigated the four most important GridWorlds in the lab, but we have also created some optional worlds (numbers 5 to 7) which you can try to solve. There is also World 8, but that is a special case, so scroll down a few cells if you are interested. Here is a brief description of World 5 to 7:
- World 5, Warpspace: As the name suggests, in this world there is one tile in which the agent enters warpspace and imediatly moves to another specific location. How do you think this will affect the learning?
- World 6, Torus: In this world, the opposite edges are connected together like a rolled-up paper. If you connect both the up-down and left-right edges, you get a mathematical shape called a torus which has no edges. This means that the closest path to the goal might not be obvious anymore.
- World 7, Steps: This world is a staircase of increasing rewards (although still all negative). However, moving up the stairs towards higher rewards also puts the agent further from the goal. So what is the optimal choice, to go for the long path with higher rewards, or to sprint throught the low rewards towards the goal. This depends on the value of the discount factor.

In [ ]:
WOpt = GridWorld( ??? )
QOpt = QLearning(WOpt, {"LearningRate": 0.99, "DiscountFactor": 0.9, "ExplorationRate": 0.9, "Epochs": 1000, "MaxSteps": 200, "DrawInterval": 100})

In [ ]:
QLearningTest(W=WOpt, Q=QOpt, params={"Epochs": 5, "MaxSteps": 100})

### 5.1: World 8

So far, every world has been a 2D-grid (y and x dimensions), and the four actions have been the same in every world. In this world however, we extend the grid to 3 dimensions, and add actions to move up or down between floors. Amazingly, if you floolowed the instruction for how to define and index your Q-table, this will work without modifications! Let's try it in World 8, where the agent has the choice of moving between two floors of the map. This is shown as diagonal up or diagonal down arrows.

In [ ]:
W8 = GridWorld(8)
Q8 = QLearning(W8, {"LearningRate": 0.99, "DiscountFactor": 0.9, "ExplorationRate": 0.9, "Epochs": 1000, "MaxSteps": 200, "DrawInterval": 100})

In [ ]:
QLearningTest(W=W8, Q=Q8, params={"Epochs": 5, "MaxSteps": 100})

---
## 6. Rocket hovering

The final optional world is not a GridWorld at all, but instead a simple rocket hovering task. The goal of the Q-learner is to keep the rocket hovering as close to the center of the screen as possible (inside the marked square). The rocket can fire either it's left, right, or center thrusters, or none. The state space is continuous, meaning that the position, angle, velocity, and angular velocity of the rocket can take any value. To handle this within the current framework, the state space has been discretized, with more granular bins near the critical center position to allow for more precise control when close to the goal. This means that the final state space is now 6-dimensional, representing x and y position, x and y velocity, angle, and angular velocity.

You can run the following code to start an interactive simulation where you can manually control the rocket using the keyboard. See if you can keep it hovering in the center.

Control are as follows:
- Left arrow (or 'a'): Turn left (fire right thruster)
- Right arrow (or 'd'): Turn right (fire left thruster)
- Up arrow (or 'w'): Fire center thruster
- Spacebar: Start new episode after crashing into the borders
- 'q': Quit the simulation

<font color="#FF4040">**Important:**</font> After you start the simulation, make sure to click outside the code editor (for example on the desktop) to prevent keypresses from being captured by the code editor. We admit this is not ideal, but this entire simulation is a bit of a hack to make it work within the notebook framework.

When you are satisifed, continue to the next cell where you can train a Q-learner to do the same task.

In [ ]:
import keyboard
import time
from rocketworld import RocketWorld

W_Rocket = RocketWorld()

while not keyboard.is_pressed('q'):
    W_Rocket.init()

    while not keyboard.is_pressed('q'):
        upPressed = keyboard.is_pressed('w') or keyboard.is_pressed('up')
        leftPressed = keyboard.is_pressed('a') or keyboard.is_pressed('left')
        rightPressed = keyboard.is_pressed('d') or keyboard.is_pressed('right')

        match (leftPressed, rightPressed, upPressed):
            case (False, False, False):
                action = "N"
            case (True, False, False):
                action = "R"
            case (False, True, False):
                action = "L"
            case (False, False, True):
                action = "C"
            case (True, True, False):
                action = "N"
            case (True, False, True):
                action = "R"
            case (False, True, True):
                action = "L"
            case (True, True, True):
                action = "C"

        W_Rocket.doAction(action)
        W_Rocket.draw(sleepTime=0.01)

        _, isTerm = W_Rocket.getState()
        # Check if gqoal
        if isTerm:
            break

    while not (keyboard.is_pressed('q') or keyboard.is_pressed('space')):
        time.sleep(0.01)
        pass

The cool thing about the Q-learning implementation is that it is completely environment agnostic, meaning it does not depend on the specific details of the world as long as it adheres to the same interface. And the rocket world does exactly that! This means you can use your existing Q-learning code without any modifications to train the rocket to hover. You will probably need to increase the number of epochs significantly compared to the GridWorlds, since this is a more complex task. We suggest starting with 100,000 epochs, but even this might not be enough. If you only want to run the training without having to do the fine-tuning yourself, we give suggestions for good hyperparemters below.

<details>
<summary><font color='#8080FF'>Click here for suggested hyperparameters</font></summary>

----------

We suggest the following hyperparameters, which we have found to work reasonably well. Note that our solution implements exploration rate decay which gradually reduces the exploration rate over time to allow for more fine tuning of the learned policy in the later stages of training. If you do not have this, you probably want to set a lower initial exploration rate.

```python
params = {
    "LearningRate": 0.1,
    "DiscountFactor": 0.99,
    "ExplorationRate": 0.99,
    "Epochs": 100000,
    "MaxSteps": 250,
    "DrawInterval": 10000
}
```
----------
</details>

In [ ]:
from rocketworld import RocketWorld
W_Rocket = RocketWorld()
Q_Rocket = QLearning(W_Rocket, {"LearningRate": ???, "DiscountFactor": ???, "ExplorationRate": ???, "Epochs": 100000, "MaxSteps": 250, "DrawInterval": 10000})

In [ ]:
QLearningTest(W=W_Rocket, Q=Q_Rocket, params={"Epochs": 20, "MaxSteps": 1000, "DrawState": True})